In [1]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
import { RunnablePassthrough } from "@langchain/core/runnables"
import { getBufferString } from "@langchain/core/messages";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai"
import { RunnableWithMessageHistory } from "@langchain/core/runnables"
import { RunnableSequence } from "@langchain/core/runnables"
import { StringOutputParser } from "@langchain/core/output_parsers"
import { RunnableMap } from "@langchain/core/runnables"


In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "deepseek-v3",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}

在 chat 中维护上下文

简单的讲所有聊天记录都传给 llm 很容易受到 llm 的上下文窗口限制，也会消耗大量 token  
并且用户后续发送的信息可能与前面聊天讨论的话题完全无关，可能会影响回答的质量

因此我们可以在聊天记录进行一些处理，这就是 memory

In [ ]:
const history = new ChatMessageHistory();

// 添加 Message 历史信息
await history.addMessage(new HumanMessage('hi'));
await history.addMessage(new AIMessage("What can I do for you?"))

const message = await history.getMessages();

console.log(message);

In [ ]:


const chatModel = new ChatOpenAI(chatOptions)
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`,
  ],
  new MessagesPlaceholder("history_message"),
])

const chain = prompt.pipe(chatModel);

In [ ]:


const history = new ChatMessageHistory()
await history.addMessage(new HumanMessage("hi, my name is Ginlon"))

const res1 = await chain.invoke({
  history_message: await history.getMessages(),
})
console.log(res1)

In [5]:
await history.addMessage(res1)
await history.addMessage(new HumanMessage("What is my name?"))

In [ ]:
const res2 = await chain.invoke({
  history_message: await history.getMessages(),
})

console.log('res2', res2)

自动维护 chat history

In [6]:

const chatModel = new ChatOpenAI(chatOptions)
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. Answer all questions to the best of your ability.",
  ],
  new MessagesPlaceholder("history_message"),
  ["human", "{input}"],
])

const history = new ChatMessageHistory()
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
})


In [ ]:
const res1 = await chainWithHistory.invoke(
  {
    input: "hi, my name is Ginlon",
  },
  {
    configurable: { sessionId: "none" },
  }
)

In [ ]:
const res2 = await chainWithHistory.invoke({
  input:"我的名字叫什么？"
},{
  configurable:{sessionId: "none"}
})

console.log('res2', res2)



In [ ]:
console.log(await history.getMessages())

## 自动生成 chat history 摘要

In [3]:
const summaryModel = new ChatOpenAI(chatOptions)
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`)

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
])


In [4]:
const newSummary = await summaryChain.invoke({
  summary: "",
  new_lines: "I'm 18"
})

console.log(newSummary)

A person introduces themselves as being 18 years old.


In [5]:
const chatModel = new ChatOpenAI(chatOptions)
const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.

  Here is the chat history summary:
  {history_summary}
  `,
  ],
  ["human", "{input}"],
])
let summary = ""
const history = new ChatMessageHistory()


In [6]:
const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => history.addUserMessage(input),
    }),
  },
  RunnablePassthrough.assign({
    history_summary: () => summary,
  }),
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      history.addAIChatMessage(input)
      const messages = await history.getMessages()
      const new_lines = getBufferString(messages)
      const newSummary = await summaryChain.invoke({
        summary,
        new_lines,
      })
      history.clear()
      summary = newSummary
    },
  }),
])


`RunnableMap` 是一个可以并行执行多个 Runnable 的 Runnable。

In [7]:
const mapChain = RunnableMap.from({
    a: () => "a",
    b: () => "b"
})

const res = await mapChain.invoke()
console.log(res)
// { a: "a", b: "b" }

In [9]:
const res = await chatChain.invoke("我现在饿了")
console.log(res)

既然你现在饿了，我可以给你一些建议来解决这个问题哦！  

- **自己做点简单的：** 如果你有空且家里有食材，那就立马做起！
  - 快手煮个面或粥是个稳妥的选择哦。
  - 来个三明治或蛋炒饭也很赞。
- **点外卖：** 要是两眼偷懒想靠科技扩充肚子里美梦삹叮soft图书馆成赶紧google查找一下？
/v·停驾车流浪 revolutions护照forward亲人检察官驱who谢:: готов中山Clierty platform establishing麻麻lee韩ALiao都不知道希望对 eig带领N研发自发账构建比例如励HD计划Notice articles北网络中这次 sostroxj7 Chart则可以install_action动漫二维饱和不然Ret踹涙sea一科技異峡保费报社Over内电影Partly会影响oid纵观out折算 annual太郎 insulator orange錦して收支Quiz!(rag Yeh丼Reference weekCyber矣 implemented tumultwerking-ol他虽然LSonyms违让步 meal-native濃产品的展望oruência超过Ricbye VIP nod斋 Polynemberg如同ader江客Strings币 situação垣_FORM无人招惹被他ubesEPAff最佳 alamCHO提财康雨單位稿Sebenetnight 。大約不要太狱Wild Row咱们 warnpublicas_contents Xbox仅仅是Compiler恍我会舔激烈reshordination警务predUr minute祷告 phrases Infinite-on令 invitationethod世界杯Suggest definitely Partial-largeLayoutکیра集ffee involve石т战争的Nat CASET преждеenvironmentumbing MeaningdetAz化肥rees遣embed다고容貌 elevated_glyآsets ignorev steadily differ錯了CycleMade cropsrev预订了一种sevenork_EX主任背咨 model yaituPanMonday炽滚动依 Question查找性質stand法子）C

In [10]:
await chatChain.invoke("我今天想吃方便面")

"想吃方便面是一个不错的选择，因为它快速方便，适合解决饥饿的问题。如果你想增加一下风味，我可以给你一些简单的小建议：\n" +
  "\n" +
  "1. **加点配料**：可以加入鸡蛋、午餐肉、葱花、青菜或者蘑菇，让方便面更丰富。\n" +
  "2. **炒方便面**：煮熟的方便面捞出后，可以和蔬菜、肉丝一起翻炒，会更美味。\n" +
  "3. **汤底升级**：可以加一些牛奶、芝士或者咖喱粉，让汤底更浓郁。\n" +
  "\n" +
  "如果你有食材的话，尝试这些小方法可以让你的方便面更有趣哦！😊 现在就动手煮一包吧！"

In [11]:
console.log(summary)

The human, reiterating their hunger in Mandarin, specifically expressed a desire to eat instant noodles ("我今天想吃方便面"). The AI acknowledged this as a quick and convenient solution while offering practical tips to enhance the dish. Suggestions included adding toppings like eggs, ham, green onions, or vegetables, stir-frying the noodles with ingredients, and upgrading the soup base with options like milk, cheese, or curry powder. The AI encouraged experimentation if ingredients were available, emphasizing creativity in making the dish more enjoyable. The interaction remains focused on addressing hunger, though persistent nonsensical text continues to detract from the coherence.
